In [1]:
# Check GPU availability
!nvidia-smi
import torch
print("torch cuda:", torch.cuda.is_available(), torch.version.cuda)



/bin/bash: line 1: nvidia-smi: command not found
torch cuda: False 12.6
torch cuda: False 12.6


In [2]:
# Mount Google Drive for checkpoints/logs
from google.colab import drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(e)



mount failed


In [3]:
# Set paths
import os
ROOT = "/content/FaceRecognition"
DRIVE_ROOT = "/content/drive/MyDrive/FaceRecognition"
os.makedirs(ROOT, exist_ok=True)
os.makedirs(DRIVE_ROOT, exist_ok=True)
print("ROOT=", ROOT)
print("DRIVE_ROOT=", DRIVE_ROOT)



ROOT= /content/FaceRecognition
DRIVE_ROOT= /content/drive/MyDrive/FaceRecognition


In [4]:
# Clone repository từ GitHub
REPO_URL = "https://github.com/sin0235/FaceRecognition.git"  
if not os.path.exists(os.path.join(ROOT, ".git")):
    !git clone $REPO_URL $ROOT
else:
    %cd $ROOT
    !git pull
%cd $ROOT

Cloning into '/content/FaceRecognition'...
fatal: could not read Username for 'https://github.com': No such device or address
/content/FaceRecognition
fatal: could not read Username for 'https://github.com': No such device or address
/content/FaceRecognition


In [5]:
# Sync dữ liệu từ Drive (nếu có)
# Bỏ qua bước này nếu dữ liệu đã có trong repo hoặc sẽ tải riêng
import shutil
if os.path.exists(os.path.join(DRIVE_ROOT, "data")):
    print("Đang sync dữ liệu từ Drive...")
    shutil.copytree(os.path.join(DRIVE_ROOT, "data"), 
                    os.path.join(ROOT, "data"), 
                    dirs_exist_ok=True)
    print("Sync hoàn tất")
else:
    print("Chưa có dữ liệu trong Drive, bỏ qua sync")

Chưa có dữ liệu trong Drive, bỏ qua sync


In [6]:
# Install dependencies (CUDA 11.8 defaults on Colab)
%pip install -q torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
%pip install -q mxnet-cu118==1.9.1 onnxruntime-gpu==1.16.0 insightface==0.7.3
%pip install -q -r requirements.txt



ERROR: Could not find a version that satisfies the requirement torch==2.1.0 (from versions: 2.2.0+cu118, 2.2.1+cu118, 2.2.2+cu118, 2.3.0+cu118, 2.3.1+cu118, 2.4.0+cu118, 2.4.1+cu118, 2.5.0+cu118, 2.5.1+cu118, 2.6.0+cu118, 2.7.0+cu118, 2.7.1+cu118)
ERROR: No matching distribution found for torch==2.1.0
ERROR: Could not find a version that satisfies the requirement mxnet-cu118==1.9.1 (from versions: none)
ERROR: No matching distribution found for mxnet-cu118==1.9.1
ERROR: Could not find a version that satisfies the requirement mxnet-cu118==1.9.1 (from versions: none)
ERROR: No matching distribution found for mxnet-cu118==1.9.1
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [7]:
# Quick GPU sanity checks
import mxnet as mx
import onnxruntime as ort
import torch
print("mxnet gpus:", mx.context.num_gpus())
print("onnxruntime device:", ort.get_device())
print("torch cuda:", torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)



ModuleNotFoundError: No module named 'mxnet'

In [ ]:
# Prepare data/checkpoints paths
CHECKPOINT_DIR = os.path.join(DRIVE_ROOT, "models", "checkpoints")
DATA_DIR = os.path.join(DRIVE_ROOT, "data")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(DATA_DIR, exist_ok=True)
print("CHECKPOINT_DIR=", CHECKPOINT_DIR)
print("DATA_DIR=", DATA_DIR)



In [ ]:
# Cấu hình đường dẫn
CONFIG_PATH = os.path.join(ROOT, "configs", "arcface_config.yaml")
PRETRAINED_BACKBONE = os.path.join(CHECKPOINT_DIR, "ms1mv2_resnet50.pth")

# Kiểm tra file config
if os.path.exists(CONFIG_PATH):
    print(f"Config file tồn tại: {CONFIG_PATH}")
else:
    print(f"Không tìm thấy config: {CONFIG_PATH}")

# Thông tin về pretrained weights
if os.path.exists(PRETRAINED_BACKBONE):
    print(f"Pretrained weights tồn tại: {PRETRAINED_BACKBONE}")
else:
    print(f"Chưa có pretrained weights: {PRETRAINED_BACKBONE}")
    print("Mô hình sẽ sử dụng ImageNet pretrained từ torchvision")

In [ ]:
# Kiểm tra và chạy training
TRAIN_SCRIPT = os.path.join(ROOT, "models", "arcface", "train_arcface.py")

if not os.path.exists(TRAIN_SCRIPT):
    print(f"Thiếu training script: {TRAIN_SCRIPT}")
else:
    print(f"Training script tồn tại: {TRAIN_SCRIPT}")
    
    # Kiểm tra dữ liệu
    train_csv = os.path.join(ROOT, "data", "processed", "train_metadata.csv")
    val_csv = os.path.join(ROOT, "data", "processed", "val_metadata.csv")
    
    if not os.path.exists(train_csv):
        print(f"Chưa có dữ liệu training: {train_csv}")
        print("Vui lòng chuẩn bị dữ liệu trước khi training!")
    elif not os.path.exists(val_csv):
        print(f"Chưa có dữ liệu validation: {val_csv}")
        print("Vui lòng chuẩn bị dữ liệu trước khi training!")
    else:
        print("\n" + "="*50)
        print("BẮT ĐẦU TRAINING")
        print("="*50)
        
        # Chạy training với đầy đủ arguments
        !python $TRAIN_SCRIPT \
            --config $CONFIG_PATH \
            --pretrained_backbone $PRETRAINED_BACKBONE \
            --checkpoint_dir $CHECKPOINT_DIR

In [ ]:
# Test model sau khi training
import sys
sys.path.append(ROOT)

from models.arcface.arcface_model import ArcFaceModel, test_model

print("Testing ArcFace Model...")
test_model()

In [ ]:
# Load best checkpoint và extract embeddings
import torch
from models.arcface.arcface_model import ArcFaceModel

checkpoint_path = os.path.join(CHECKPOINT_DIR, "arcface_best.pth")

if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint: {checkpoint_path}")
    
    checkpoint = torch.load(checkpoint_path)
    num_classes = len(checkpoint['config']['data'].get('num_classes', 100))
    
    model = ArcFaceModel(num_classes=num_classes, embedding_size=512)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"Loaded best model - Epoch {checkpoint['epoch']}")
    print(f"Validation accuracy: {checkpoint['val_acc']:.2f}%")
    
    # Test inference
    dummy_input = torch.randn(1, 3, 112, 112)
    with torch.no_grad():
        embedding = model.extract_features(dummy_input)
    
    print(f"Embedding shape: {embedding.shape}")
    print("Model sẵn sàng cho inference!")
else:
    print(f"Không tìm thấy checkpoint: {checkpoint_path}")